In [1]:
# imports
import numpy as np
import pandas as pd
import os
import requests
import json

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
# import langitude and latitude coordinates from Part 1 City bike acitivity and store in a Dataframe
city_bike_coord = pd.read_csv('../data/CityBike_longlat.csv')
city_bike_coord = city_bike_coord.round(2)
city_bike_coord

In [ ]:
# get each pair of long lat coordinates
long_lat_strings = []

for index, row in city_bike_coord.iterrows():
  # Extract the longitude and latitude values
  longitude = row['longitude']
  latitude = row['latitude']

  # Combine them into a tuple
  long_lat_pair = f"{longitude},{latitude}"

  long_lat_strings.append(long_lat_pair)

long_lat_strings

In [ ]:
# API URL and API token stored as an environment variable
client_access_token = os.environ['FOURSQUARE_AUTH_TOKEN']

# get response in json while passing API token in the header
headers = {
    "Accept": "application/json",
    "Authorization": client_access_token
}

responses = []

for long_lat_string in long_lat_strings:
    #URL
    
    longitude, latitude = long_lat_string.split(",")
    url = f"https://api.foursquare.com/v3/places/search?ll={latitude},{longitude}&radius=1000&query=restaurant"
    response = requests.get(url, headers=headers)
    data = response.json()
    responses.append(data)

print(responses)


In [ ]:
# Flatten responses by two levels to get results
flat_responses=[]
for level_one in responses:
    for level_two in level_one['results']:
        flat_responses.append(level_two)
flat_responses



Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [6]:
# create a new dataframe we will append to
results =[]


#iterate through results "data['results]" will give iterate through json data from one level down
for result in flat_responses:
    name = result['name']
    location = result['location']

    categories = []
    for category in result['categories']:
        categories.append(category['name'])

    result_dict = {
        'name': name,
        'location': location,
        'categories': categories
    }
    # add respective POI in list object
    results.append(result_dict)

Put your parsed results into a DataFrame

In [7]:
# store results in dataframe
df = pd.DataFrame(results)
df_name = df['name']
df_location = pd.json_normalize(df['location'])
df_categories = df['categories']

merged_df = pd.merge(df_name, df_location,left_index=True, right_index=True,how='inner')
final_merge_df = pd.merge(merged_df, df_categories,left_index=True, right_index=True, how='inner')
final_merge_df

,name,address,country,cross_street,formatted_address,locality,postcode,region,address_extended,categories
0,La Luna Restaurant Downtown,306 King St W,CA,at Queen St,"306 King St W (at Queen St), Hamilton ON L8P 1B1",Hamilton,L8P 1B1,ON,NaN,"[Lebanese Restaurant, Mediterranean Restaurant..."
1,Coop Hamilton,274 King St W,CA,,"274 King St W, Hamilton ON L8P 1J6",Hamilton,L8P 1J6,ON,# 272,[Fried Chicken Joint]
2,Smoke's Poutinerie,112 George St,CA,at Queen St. S.,"112 George St (at Queen St. S.), Hamilton ON L...",Hamilton,L8P 1E2,ON,NaN,"[Fast Food Restaurant, Poutine Restaurant, Ste..."
3,Coop Wicked Chicken Hamilton,274 King St W,CA,Hess St,"274 King St W (Hess St), Hamilton ON L8P 1J6",Hamilton,L8P 1J6,ON,# 272,[Fried Chicken Joint]
4,Vida la Pita,217 King St W,CA,Caroline,"217 King St W (Caroline), Hamilton ON L8P 1A7",Hamilton,L8P 1A7,ON,NaN,"[Fast Food Restaurant, Middle Eastern Restaurant]"
...,...,...,...,...,...,...,...,...,...,...
1336,Hambrgr,207 Ottawa St N,CA,,"207 Ottawa St N, Hamilton ON L8H 3Z4",Hamilton,L8H 3Z4,ON,NaN,"[Bar, Burger Joint]"
1337,Cannon Coffee Co,179 Ottawa St N,CA,Cannon Street,"179 Ottawa St N (Cannon Street), Hamilton ON L...",Hamilton,L8H 3Z4,ON,NaN,"[Café, Coffee Shop, Restaurant]"
1338,Boston Pizza,1219 Barton St E,CA,Kenilworth Ave. N.,"1219 Barton St E (Kenilworth Ave. N.), Hamilto...",Hamilton,L8H 2V4,ON,NaN,[Pizzeria]
1339,Hi-Line Centre Pizza & Wings,1150 King St E,CA,Gage,"1150 King St E (Gage), Hamilton ON L8M 1E8",Hamilton,L8M 1E8,ON,# 2,[Pizzeria]


In [104]:
final_merge_df.to_csv('Foursquare_Biz_Data.csv', index=False)

In [105]:
%%bash
mv Foursquare_Biz_Data.csv ../data/ # Move csv file to data folder

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [8]:
# API URL and API token stored as an environment variable
yelp_access_token = os.environ['YELP_AUTH_TOKEN']

# get response in json while passing API token in the header
headers = {
    "Accept": "application/json",
    "Authorization": f'Bearer {yelp_access_token}'
}



yelp_responses = []


for long_lat_string in long_lat_strings:
    

    longitude, latitude = long_lat_string.split(",")
    url = f"https://api.yelp.com/v3/businesses/search?latitude={latitude}&longitude={longitude}&radius=1000"
    response = requests.get(url, headers=headers)
    data = response.json()
    yelp_responses.append(data)

print(yelp_responses)

[{'businesses': [{'id': 'vqyK2q3zJ74TIT1-7Bf3Tg', 'alias': 'la-luna-hamilton', 'name': 'La Luna', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/OBRpyj_wUqmDwKLtKU4pCA/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/la-luna-hamilton?adjust_creative=JKSqwVDqPT5eEI-KtMB0CA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=JKSqwVDqPT5eEI-KtMB0CA', 'review_count': 66, 'categories': [{'alias': 'mideastern', 'title': 'Middle Eastern'}], 'rating': 3.8, 'coordinates': {'latitude': 43.25942165323912, 'longitude': -79.87848757635106}, 'transactions': [], 'price': '$$', 'location': {'address1': '306 King Street W', 'address2': '', 'address3': '', 'city': 'Hamilton', 'zip_code': 'L8P 1B1', 'country': 'CA', 'state': 'ON', 'display_address': ['306 King Street W', 'Hamilton, ON L8P 1B1', 'Canada']}, 'phone': '+19055770233', 'display_phone': '+1 905-577-0233', 'distance': 138.33065443619196, 'attributes': {'business_temp_closed': None, 'menu_url': '', 'open24_ho

In [ ]:
# Flatten and go down two levels
flat_yelp_response = []
for i in range(len(yelp_responses)):
    for level_two in yelp_responses[i]['businesses']:
        flat_yelp_response.append(level_two)
flat_yelp_response

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
# Get the respective POI data
names = []
ratings =[]
pricing = []
rev_count = []
address = []
is_closed = []
phone_num = []
coordinates = []
for i in range(len(flat_yelp_response)):
    names.append(flat_yelp_response[i]['name'])
    ratings.append(flat_yelp_response[i]['rating'])
    address.append(flat_yelp_response[i]['location']['address1'])
    rev_count.append(flat_yelp_response[i]['review_count'])
    is_closed.append(flat_yelp_response[i]['is_closed'])
    phone_num.append(flat_yelp_response[i]['phone'])
    coordinates.append(flat_yelp_response[i]['coordinates'])

    try:
        pricing.append(len(flat_yelp_response[i]['price']))
    except:
        pricing.append(None)

yelp_business = {'name': names, 'rating': ratings, 'pricing': pricing, 'review_count': rev_count, 'street address': address, 'Is Closed': is_closed, 'Contact#': phone_num, 'Coordinates': coordinates}
yelp_business

Put your parsed results into a DataFrame

In [ ]:
# Create dataframe
yelp_biz_df = pd.DataFrame(yelp_business)
yelp_biz_df

In [ ]:
# Split Coordinates in deparate DF
biz_coord_dict = yelp_biz_df['Coordinates'].to_dict().values()
biz_coord_df = pd.DataFrame(biz_coord_dict).round(2)
biz_coord_df


In [ ]:
# Get remaining df elements, minus Coordinates
non_coord_biz_data_df = yelp_biz_df[['name','rating','pricing','review_count','street address','Is Closed','Contact#']]
#non_coord_biz_data_df = pd.DataFrame(non_coord_biz_data)
non_coord_biz_data_df


In [55]:
#merge coords and df
merged_yelpBizDataWithCoord = pd.merge(non_coord_biz_data_df, biz_coord_df, left_index=True, right_index=True, how='inner')
merged_yelpBizDataWithCoord


,name,rating,pricing,review_count,street address,Is Closed,Contact#,latitude,longitude
0,La Luna,3.8,2.0,66,306 King Street W,False,+19055770233,43.26,-79.88
1,Bardō Locke Street,4.3,2.0,293,258 Locke Street S,False,+19055222999,43.25,-79.89
2,Pho Nhung,4.4,NaN,18,85 Queen St N,False,+19055258008,43.26,-79.88
3,The Ship,4.1,2.0,207,23 Augusta Street,False,+19055260792,43.25,-79.87
4,The Coop,3.8,2.0,43,274-274 King Street W,False,+12897782667,43.26,-79.88
...,...,...,...,...,...,...,...,...,...
2798,Gage Park Diner,4.1,2.0,7,975 Main Street E,False,+12893898333,43.25,-79.83
2799,The Parlour Dairy Bar,4.5,NaN,2,224 Ottawa St N,False,+12893895552,43.25,-79.82
2800,Willard’s Ice Cream,4.7,NaN,7,942 Main St E,False,,43.25,-79.83
2801,Mike's Subs,3.8,1.0,9,122 Ottawa Street N,False,+19055447182,43.25,-79.82


In [58]:
# export dataframe to CSV
merged_yelpBizDataWithCoord.to_csv('Yelp_Restaurant_Data.csv', index=False)

In [59]:
%%bash
mv Yelp_Restaurant_Data.csv ../data/ # Move csv file to data folder

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp gave more complete data than Foursquare. See reasons below
- You had access to ratings for the respective restaurant
- You could also see how many reviews that restaurant received to measure rating confidence
- You also get restaurant contact information
Overall the Yelp API gives significantly more detail than the Foursquare API, allowing users to make more informed decisions on where to visit based on wide array of optons

Get the top 10 restaurants according to their rating

In [60]:
sorted_df = merged_yelpBizDataWithCoord.sort_values(by='rating', ascending=False)
top_10_rest_by_rating = sorted_df.head(10)
top_10_rest_by_rating

,name,rating,pricing,review_count,street address,Is Closed,Contact#,latitude,longitude
1401,Saha,5.0,NaN,1,28 Eastbourne Avenue,False,+14168010995,43.25,-79.84
988,Golden Tenders,5.0,NaN,1,1150 King Street E,False,+13653668108,43.25,-79.83
1061,The Indian Connection Pizza,5.0,NaN,2,1022 Barton Street E,False,,43.25,-79.82
1052,Pita Pit,5.0,NaN,1,49 King Street E,False,+12892388282,43.27,-79.95
1049,Little Caesars,5.0,NaN,1,1685 Main Street W,False,+19055274884,43.26,-79.93
1043,Sio Shokudo,5.0,NaN,2,1686 Main Street W,False,+16478725562,43.26,-79.93
1030,Little Caesars,5.0,NaN,1,1685 Main Street W,False,+19055274884,43.26,-79.93
1023,Sio Shokudo,5.0,NaN,2,1686 Main Street W,False,+16478725562,43.26,-79.93
1011,Little Caesars,5.0,NaN,1,1685 Main Street W,False,+19055274884,43.26,-79.93
1008,CoCo Fresh Tea & Juice,5.0,NaN,1,1548 Main Street W,False,+12893960888,43.26,-79.93
